In [21]:
import os
os.environ["DEEPLAKE_DOWNLOAD_PATH"] = "./dataset"

In [22]:
import deeplake
from torchvision import transforms
from pprint import pprint
import numpy as np

ds = deeplake.dataset('hub://activeloop/wiki-art', access_method = 'local')

# filtered_labels = ['realism', 'impressionism', 'romanticism'] #? Top Three
filtered_labels = ["action_painting", "analytical_cubism", "synthetic_cubism"] #? Bottom three, for testing
@deeplake.compute
def filter_labels(sample_in, labels_list):
    return sample_in.labels.data()['text'][0] in labels_list

ds_view = ds.filter(filter_labels(filtered_labels), scheduler = 'threaded', num_workers = 0)
first_eight_percent = int(len(ds_view) * 0.8)
train = ds_view[:first_eight_percent]
validate = ds_view[first_eight_percent:]

print("Unfiltered len:", len(ds))
print("Filtered len:", len(ds_view))
print("Train:", len(train))
print("Validate:", len(validate))

transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5322, 0.4754, 0.4146),
                         (0.2907, 0.2803, 0.2766)),

    # TODO: figure out a reasonable value here
    transforms.CenterCrop(32)
])

wikiart_train = train.pytorch( 
    transform={'images': transformation, 'labels': None},
    tensors=["images", "labels"],
    # decode_method={'images':'pil', 'labels': 'numpy'},
    shuffle=True,
    batch_size=32
)

wikiart_validate = validate.pytorch( 
    transform={'images': transformation, 'labels': None},
    tensors=["images", "labels"],
    # decode_method={'images':'pil', 'labels': 'numpy'},
    shuffle=True,
    batch_size=32
)

./dataset\hub_activeloop_wiki-art loaded successfully.
** Loaded local copy of dataset. Downloaded on: Wed Dec 14 00:48:59 2022


100%|██████████| 81433/81433 [00:02<00:00, 28445.17it/s]


Unfiltered len: 81433
Filtered len: 424
Train: 339
Validate: 85


In [23]:
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 27)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

class Net_Skip(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 3)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.tanh(self.conv3(out)) + out1, 2)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


class Net_Dropout(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)
        self.conv3_dropout = nn.Dropout2d(p=0.3)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 3)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = self.conv3_dropout(out)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


def training_function(train_loader, model, loss_fn, optimizer, device, n_epochs):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            labels = labels.flatten() # this is needed because for some reason every label is in a seperate tensor
            # print(labels)
            # print(imgs.shape)
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs).to(device=device)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


def validate(model, train_loader, val_loader, device):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                # labels = labels.flatten()
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs).to(device=device)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        print("Accuracy {}: {:.6f}".format(name, correct / total))

In [24]:
import torch
import torch.optim as optim

device = (torch.device('cuda') if torch.cuda.is_available()
                      else torch.device('cpu'))

cnn32 = Net(32)

print(f"Training on {device}:")
print(f"\tdevice: {torch.cuda.get_device_name(0)}")
print("\tSize of wikiart:", len(wikiart_train))

training_function(
    train_loader = wikiart_train,
    model = cnn32.to(device=device),
    loss_fn = nn.NLLLoss(),
    optimizer = optim.SGD(cnn32.parameters(), lr=1e-2),
    device=device,
    n_epochs = 1,
)

validate(cnn32.to(device=device), wikiart_train, wikiart_validate, device)

Training on cuda:
	device: NVIDIA GeForce RTX 3060 Ti
	Size of wikiart: 11


Please wait, filling up the shuffle buffer with samples.:  37%|███▋      | 720M/1.91G [00:04<00:07, 171MB/s]  


Shuffle buffer filling is complete.
2022-12-16 10:26:37.125014 Epoch 1, Training loss 3.1730652072212915


Please wait, filling up the shuffle buffer with samples.:  37%|███▋      | 720M/1.91G [00:04<00:07, 175MB/s]  


Shuffle buffer filling is complete.
196 32
486 64
695 96
1107 128
1307 160
1692 192
1976 224
2200 256
2522 288
2766 320
2886 339
Accuracy train: 8.513274


Please wait, filling up the shuffle buffer with samples.:  11%|█         | 212M/1.91G [00:02<00:20, 89.5MB/s] 


Shuffle buffer filling is complete.
0 32
0 64
0 85
Accuracy val: 0.000000
